# Deploy and predict with Keras model on Cloud AI Platform.

**Learning Objectives**

1. Setup up the environment
1. Deploy trained Keras model to Cloud AI Platform
1. Online predict from model on Cloud AI Platform
1. Batch predict from model on Cloud AI Platform


## Introduction 
**Verify that you have previously Trained your Keras model. If not, go back to [train_keras_ai_platform_babyweight.ipynb](../solutions/train_keras_ai_platform_babyweight.ipynb) create them.**
In this notebook, we'll be deploying our Keras model to Cloud AI Platform and creating predictions.

We will set up the environment, deploy a trained Keras model to Cloud AI Platform, online predict from deployed model on Cloud AI Platform, and batch predict from deployed model on Cloud AI Platform.

Each learning objective will correspond to a __#TODO__ in this student lab notebook.

## Set up environment variables and load necessary libraries

Import necessary libraries.

In [ ]:
!sudo chown -R jupyter:jupyter /home/jupyter/training-data-analyst

In [ ]:
import os

### Set environment variables.

Set environment variables so that we can use them throughout the entire lab. We will be using our project name for our bucket, so you only need to change your project and region.

In [ ]:
%%bash
PROJECT=$(gcloud config list project --format "value(core.project)")
echo "Your current GCP Project Name is: "$PROJECT

Your current GCP Project Name is: qwiklabs-gcp-03-596373e6e715


In [ ]:
# Change these to try this notebook out
BUCKET = 'qwiklabs-gcp-03-596373e6e715'
PROJECT = 'qwiklabs-gcp-03-596373e6e715'
REGION = 'us-central1' # TODO 1 Replace with your REGION

In [ ]:
os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION
os.environ["TFVERSION"] = "1.8"

In [ ]:
%%bash
gcloud config set compute/region $REGION

Updated property [compute/region].


## Check our trained model files

Let's check the directory structure of our outputs of our trained model in folder we exported the model to in our last [lab](../solutions/10_train_keras_ai_platform_babyweight.ipynb). We'll want to deploy the saved_model.pb within the timestamped directory as well as the variable values in the variables folder. Therefore, we need the path of the timestamped directory so that everything within it can be found by Cloud AI Platform's model deployment service.

In [ ]:
%%bash
gsutil ls gs://${BUCKET}/babyweight/trained_model

gs://qwiklabs-gcp-03-596373e6e715/babyweight/trained_model/checkpoint
gs://qwiklabs-gcp-03-596373e6e715/babyweight/trained_model/events.out.tfevents.1529347276.cmle-training-master-a137ac0fff-0-9q8r4
gs://qwiklabs-gcp-03-596373e6e715/babyweight/trained_model/graph.pbtxt
gs://qwiklabs-gcp-03-596373e6e715/babyweight/trained_model/model.ckpt-342784.data-00000-of-00003
gs://qwiklabs-gcp-03-596373e6e715/babyweight/trained_model/model.ckpt-342784.data-00001-of-00003
gs://qwiklabs-gcp-03-596373e6e715/babyweight/trained_model/model.ckpt-342784.data-00002-of-00003
gs://qwiklabs-gcp-03-596373e6e715/babyweight/trained_model/model.ckpt-342784.index
gs://qwiklabs-gcp-03-596373e6e715/babyweight/trained_model/model.ckpt-342784.meta
gs://qwiklabs-gcp-03-596373e6e715/babyweight/trained_model/model.ckpt-376661.data-00000-of-00003
gs://qwiklabs-gcp-03-596373e6e715/babyweight/trained_model/model.ckpt-376661.data-00001-of-00003
gs://qwiklabs-gcp-03-596373e6e715/babyweight/trained_model/model.ckpt-376661.da

In [ ]:
%%bash
MODEL_LOCATION=$(gsutil ls -ld -- gs://${BUCKET}/babyweight/trained_model/2* \
                 | tail -1)
gsutil ls ${MODEL_LOCATION}

gs://qwiklabs-gcp-03-596373e6e715/babyweight/trained_model/20201217030149/
gs://qwiklabs-gcp-03-596373e6e715/babyweight/trained_model/20201217030149/saved_model.pb
gs://qwiklabs-gcp-03-596373e6e715/babyweight/trained_model/20201217030149/assets/
gs://qwiklabs-gcp-03-596373e6e715/babyweight/trained_model/20201217030149/variables/


## Lab Task #2: Deploy trained model

Deploying the trained model to act as a REST web service is a simple gcloud call.

In [ ]:
%%bash
MODEL_NAME="babyweight"
MODEL_VERSION="ml_on_gcp"
MODEL_LOCATION=$(gsutil ls -ld -- gs://${BUCKET}/babyweight/trained_model/2* | tail -1 | tr -d '[:space:]') # TODO 2
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION"
# gcloud ai-platform versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
# gcloud ai-platform models delete ${MODEL_NAME}
gcloud ai-platform models create ${MODEL_NAME} --regions ${REGION}
gcloud ai-platform versions create ${MODEL_VERSION} \
    --model=${MODEL_NAME} \
    --origin=${MODEL_LOCATION} \
    --runtime-version=2.1 \
    --python-version=3.7

Deleting and deploying babyweight ml_on_gcp from gs://qwiklabs-gcp-03-596373e6e715/babyweight/trained_model/20201217030149/


Using endpoint [https://ml.googleapis.com/]
Creating version (this might take a few minutes)......
................................................................................................................................................................................................................................................................................................................................................................................done.


## Use model to make online prediction.

### Python API

We can use the Python API to send a JSON request to the endpoint of the service to make it predict a baby's weight. The order of the responses are the order of the instances.

In [ ]:
from oauth2client.client import GoogleCredentials
import requests
import json

MODEL_NAME = "babyweight" # TODO 3a
MODEL_VERSION = "ml_on_gcp" # TODO 3a

token = GoogleCredentials.get_application_default().get_access_token().access_token
api = "https://ml.googleapis.com/v1/projects/{}/models/{}/versions/{}:predict" \
         .format(PROJECT, MODEL_NAME, MODEL_VERSION)
headers = {"Authorization": "Bearer " + token }
data = {
  "instances": [
    {
      "is_male": "True",
      "mother_age": 26.0,
      "plurality": "Single(1)",
      "gestation_weeks": 39
    },
    {
      "is_male": "False",
      "mother_age": 29.0,
      "plurality": "Single(1)",
      "gestation_weeks": 38
    },
    {
      "is_male": "True",
      "mother_age": 26.0,
      "plurality": "Triplets(3)",
      "gestation_weeks": 39
    },
    {
      # TODO 3a
      "is_male": "Unknown",
      "mother_age": 29.0,
      "plurality": "Multiple(2+)",
      "gestation_weeks": 38
    },
  ]
}
response = requests.post(api, json=data, headers=headers)
print(response.content)

b'{"predictions": [{"weight": [5.814619064331055]}, {"weight": [6.089856147766113]}, {"weight": [4.153970718383789]}, {"weight": [5.261815071105957]}]}'


The predictions for the four instances were: 5.33, 6.09, 2.50, and 5.86 pounds respectively when I ran it (your results might be different).

### gcloud shell API

Instead we could use the gcloud shell API. Create a newline delimited JSON file with one instance per line and submit using gcloud.

In [ ]:
%%writefile inputs.json
{"is_male": "True", "mother_age": 26.0, "plurality": "Single(1)", "gestation_weeks": 39}
{"is_male": "False", "mother_age": 26.0, "plurality": "Single(1)", "gestation_weeks": 39} # TODO 3b

Overwriting inputs.json


Now call `gcloud ai-platform predict` using the JSON we just created and point to our deployed `model` and `version`.

In [ ]:
%%bash
gcloud ai-platform predict \
    --model=babyweight # TODO 3b\
    --json-instances=inputs.json \
    --version=ml_on_gcp # TODO 3b

## Use model to make batch prediction.

Batch prediction is commonly used when you have thousands to millions of predictions. It will create an actual Cloud AI Platform job for prediction.

In [ ]:
%%bash
INPUT=gs://${BUCKET}/babyweight/batchpred/inputs.json
OUTPUT=gs://${BUCKET}/babyweight/batchpred/outputs
gsutil cp inputs.json $INPUT
gsutil -m rm -rf $OUTPUT 
gcloud ai-platform jobs submit prediction babypred_$(date -u +%y%m%d_%H%M%S) \
    --data-format=TEXT \
    --region ${REGION} \
    --input-paths=$INPUT \
    --output-path=$OUTPUT \
    --model=babyweight # TODO 4\
    --version=ml_on_gcp # TODO 4

## Lab Summary:
In this lab, we set up the environment, deployed a trained Keras model to Cloud AI Platform, online predicted from deployed model on Cloud AI Platform, and batch predicted from deployed model on Cloud AI Platform.